wget https://raw.githubusercontent.com/alexeygrigorev/datasets/master/housing.csv

In [34]:
import pandas as pd
import numpy as np

In [35]:
housing = pd.read_csv('/home/frog/Documents/Coding/MLZoomCamp-1/Week_2/housing.csv')

housing = housing [['longitude', 'latitude', 'housing_median_age', 'total_rooms', 'total_bedrooms', 'population', 'households', 'median_income', 'median_house_value']]

housing.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0
3,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0
4,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0


In [36]:
housing.isna().sum()

longitude               0
latitude                0
housing_median_age      0
total_rooms             0
total_bedrooms        207
population              0
households              0
median_income           0
median_house_value      0
dtype: int64

Question 1: total_bedroom has 207 missing values.

In [37]:
housing.describe()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value
count,20640.000000,20640.000000,20640.000000,20640.000000,20433.000000,20640.000000,20640.000000,20640.000000,20640.000000
mean,-119.569704,35.631861,28.639486,2635.763081,537.870553,1425.476744,499.539680,3.870671,206855.816909
std,2.003532,2.135952,12.585558,2181.615252,421.385070,1132.462122,382.329753,1.899822,115395.615874
min,-124.350000,32.540000,1.000000,2.000000,1.000000,3.000000,1.000000,0.499900,14999.000000
25%,-121.800000,33.930000,18.000000,1447.750000,296.000000,787.000000,280.000000,2.563400,119600.000000
50%,-118.490000,34.260000,29.000000,2127.000000,435.000000,1166.000000,409.000000,3.534800,179700.000000
75%,-118.010000,37.710000,37.000000,3148.000000,647.000000,1725.000000,605.000000,4.743250,264725.000000
max,-114.310000,41.950000,52.000000,39320.000000,6445.000000,35682.000000,6082.000000,15.000100,500001.000000


Question 2: population has a median value of 1166.

In [38]:
n = len(housing)

n_val = int(n * 0.2)
n_test = int(n * 0.2)
n_train = n - n_val - n_test

idx = np.arange(n)

In [39]:
np.random.seed(42)
np.random.shuffle(idx)

In [40]:
housing_train = housing.iloc[idx[:n_train]]
housing_val = housing.iloc[idx[n_train:n_train+n_val]]
housing_test = housing.iloc[idx[n_train+n_val:]]

In [41]:
housing_train = housing_train.reset_index(drop=True)
housing_val = housing_val.reset_index(drop=True)
housing_test = housing_test.reset_index(drop=True)

In [42]:
y_train = np.log1p(housing_train['median_house_value'].values)
y_val = np.log1p(housing_val['median_house_value'].values)
y_test = np.log1p(housing_test['median_house_value'].values)

In [43]:
del housing_train['median_house_value']
del housing_val['median_house_value']
del housing_test['median_house_value']

In [44]:
housing_train.isna().sum()

longitude               0
latitude                0
housing_median_age      0
total_rooms             0
total_bedrooms        207
population              0
households              0
median_income           0
dtype: int64

In [45]:
housing_test.isna().sum()

longitude             0
latitude              0
housing_median_age    0
total_rooms           0
total_bedrooms        0
population            0
households            0
median_income         0
dtype: int64

In [46]:
housing_val.isna().sum()

longitude             0
latitude              0
housing_median_age    0
total_rooms           0
total_bedrooms        0
population            0
households            0
median_income         0
dtype: int64

In [47]:
def train_linear_regression(X, y):
    ones = np.ones(X.shape[0])
    X = np.column_stack([ones, X])

    XTX = X.T.dot(X)
    XTX_inv = np.linalg.inv(XTX)
    w_full = XTX_inv.dot(X.T).dot(y)
    
    return w_full[0], w_full[1:]

In [48]:
def rmse(y, y_pred):
    se = (y - y_pred) ** 2
    mse = se.mean()
    return np.sqrt(mse)

In [49]:
housing_train_zero = housing_train.fillna(0)
housing_train_zero.isna().sum()

longitude             0
latitude              0
housing_median_age    0
total_rooms           0
total_bedrooms        0
population            0
households            0
median_income         0
dtype: int64

In [50]:
housing_train_median = housing_train.fillna(housing_train['total_bedrooms'].median())
housing_train_median.isna().sum()

longitude             0
latitude              0
housing_median_age    0
total_rooms           0
total_bedrooms        0
population            0
households            0
median_income         0
dtype: int64

In [51]:
w0_zero, w_zero = train_linear_regression(housing_train_zero, y_train)

y_pred = w0_zero + housing_val.dot(w_zero)

round(rmse(y_val, y_pred),2)

0.33

In [52]:
w0_median, w_median = train_linear_regression(housing_train_median, y_train)

y_pred = w0_median + housing_val.dot(w_median)

round(rmse(y_val, y_pred),2)

0.33

Question 3: Both are equaly good (rmse is 0.33).

In [53]:
housing_train = housing_train.fillna(0)
housing_train.isna().sum()

longitude             0
latitude              0
housing_median_age    0
total_rooms           0
total_bedrooms        0
population            0
households            0
median_income         0
dtype: int64

In [54]:
def train_linear_regression_reg(X, y, r=0.001):
    ones = np.ones(X.shape[0])
    X = np.column_stack([ones, X])

    XTX = X.T.dot(X)
    XTX = XTX + r * np.eye(XTX.shape[0])

    XTX_inv = np.linalg.inv(XTX)
    w_full = XTX_inv.dot(X.T).dot(y)
    
    return w_full[0], w_full[1:]

In [55]:
for r in [0, 0.000001, 0.0001, 0.001, 0.01, 0.1, 1, 5, 10]:
    w0, w = train_linear_regression_reg(housing_train, y_train, r=r)

    y_pred = w0 + housing_val.dot(w)
    score = round(rmse(y_val, y_pred),2)
    
    print(r, w0, score)

0 -11.686975242036546 0.33
1e-06 -11.686959175728921 0.33
0.0001 -11.685368865034667 0.33
0.001 -11.670931317799797 0.33
0.01 -11.528493585632091 0.33
0.1 -10.274500282195673 0.33
1 -4.920480897742212 0.33
5 -1.482095745621172 0.34
10 -0.7899311831779245 0.34


Question 4: all the r values result in about the same score (~0.33). Thus, r = 0.

In [58]:
housing = housing.fillna(0)
score_array = []

for i in [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]:
    np.random.seed(i)
    np.random.shuffle(idx)

    housing_train = housing.iloc[idx[:n_train]]
    housing_val = housing.iloc[idx[n_train:n_train+n_val]]
    housing_test = housing.iloc[idx[n_train+n_val:]]

    housing_train = housing_train.reset_index(drop=True)
    housing_val = housing_val.reset_index(drop=True)
    housing_test = housing_test.reset_index(drop=True)

    y_train = np.log1p(housing_train['median_house_value'].values)
    y_val = np.log1p(housing_val['median_house_value'].values)
    y_test = np.log1p(housing_test['median_house_value'].values)

    del housing_train['median_house_value']
    del housing_val['median_house_value']
    del housing_test['median_house_value']

    w0_zero, w_zero = train_linear_regression(housing_train_zero, y_train)
    y_pred = w0_zero + housing_val.dot(w_zero)
    score = rmse(y_val, y_pred)

    score_array.append(score)

round(np.std(score_array),3)

0.005

Question 5: std = 0.005.

In [63]:
np.random.seed(9)
np.random.shuffle(idx)
housing_train = housing.iloc[idx[:n_train]]
housing_val = housing.iloc[idx[n_train:n_train+n_val]]
housing_test = housing.iloc[idx[n_train+n_val:]]
housing_train = housing_train.reset_index(drop=True)
housing_val = housing_val.reset_index(drop=True)
housing_test = housing_test.reset_index(drop=True)
y_train = np.log1p(housing_train['median_house_value'].values)
y_val = np.log1p(housing_val['median_house_value'].values)
y_test = np.log1p(housing_test['median_house_value'].values)
del housing_train['median_house_value']
del housing_val['median_house_value']
del housing_test['median_house_value']
w0, w = train_linear_regression_reg(housing_train, y_train, r=r)
y_pred = w0 + housing_val.dot(w)

round(rmse(y_val, y_pred),2)


0.34

Question 6: rmse is ~0.35.